## 제 1유형

1-1. 다음은 Boston Housing 데이터셋이다. crim 항목의 상위에서 10번째 값(즉, 상위 10번째 값 중에서 가장 작은 값)으로 상위 10개의 값을 변환하고 age 80이상인 값에 대하여 crim 평균을 구하시오.

In [39]:
import pandas as pd
data = pd.read_csv('boston_housing.csv')
data_sort = data.sort_values(by = 'crim', ascending = False)
data_sort.head(10)
def change(series) :
    if series >= 25.9406 :
        return 25.9406
    else :
        return series
data_sort['crim'] = data_sort['crim'].apply(change)
#data_sort.head(10)
data_80 = data_sort[data_sort['age'] >= 80]
data_80.head(15)
print(data_80['crim'].mean())

5.759386624999999


1-2. 주어진 데이터의 첫 번째 행부터 순서대로 80%까지의 데이터를 훈련 데이터로 추출 후 'total_bedrooms' 변수의 결측값(NA)을 'total_bedrooms' 변수의 중앙값으로 대체하고, 대체 전의 'total_bedrooms' 변수 표준편차 값의 차이의 절댓값을 구하시오.

In [18]:
import pandas as pd
data = pd.read_csv('california_housing.csv')
#print(len(data))
a = data[:int(len(data)*0.8)]
a_std = a['total_bedrooms'].std()

#결측값
median = a.median()
b = a.fillna(median)
#b.isnull().sum()
b_std = b['total_bedrooms'].std()

print(abs(a_std - b_std))

1.975147291645726


1-3. 다음은 insurance 데이터셋이다. charges 항목의 이상값의 핪을 구하시오.(이상값은 평균에서 1.5표준편차 이상인 값)

In [1]:
import pandas as pd
data = pd.read_csv('insurance.csv')
mean = data['charges'].mean()
std = data['charges'].std()
#print(mean)
#print(std)

outlier = data[(data['charges'] >= mean + 1.5 * std) | (data['charges'] <= mean - 1.5 * std)]
#outlier.head()

print(outlier['charges'].sum())

6421430.0206699995


## 제 2유형

아래 E - Commerce Shipping Data의 train set을 참조하여 고객이 주문한 물품의 정시 도착 여부를 예측하시오.(ID와 예측치를 csv 파일로 저장하여 제출하시오.)

In [72]:
import pandas as pd
X_test = pd.read_csv('X_test.csv')
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')

#데이터 저장
test_id = X_test['ID']
X_test = X_test.drop(['ID'], axis = 1)
X_train = X_train.drop(['ID'], axis = 1)
y = y_train['Reached.on.Time_Y.N']

#결측치 확인
#없음

#필요없는 변수 제거
X_train = X_train.drop(['Warehouse_block'], axis = 1)
X_test = X_test.drop(['Warehouse_block'], axis = 1)


#라벨인코더(Mode_of_shipment, Product_importance, Gender)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X_train['Mode_of_Shipment'] = le.fit_transform(X_train['Mode_of_Shipment'])
X_train['Product_importance'] = le.fit_transform(X_train['Product_importance'])
X_train['Gender'] = le.fit_transform(X_train['Gender'])

X_test['Mode_of_Shipment'] = le.fit_transform(X_test['Mode_of_Shipment'])
X_test['Product_importance'] = le.fit_transform(X_test['Product_importance'])
X_test['Gender'] = le.fit_transform(X_test['Gender'])

#MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

#데이터 분할
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(X_train, y, test_size = 0.2, random_state = 49)
# print(x_train.shape, x_valid.shape, y_train.shape, y_valid.shape)

#모델링
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state = 100, n_estimators = 100, max_depth = 10, min_samples_leaf = 10)
model.fit(x_train, y_train)
predict = model.predict_proba(x_valid)

#roc
# from sklearn.metrics import roc_auc_score
# print(roc_auc_score(y_valid, predict[:,1]))

#테스트 데이터셋
predict_test = model.predict_proba(X_test)
submit = pd.DataFrame({'ID' : test_id, 'Reached.on.Time_Y.N' : predict_test[:,1]})
submit.to_csv('submission.csv', index = False)